In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [6]:
import pickle as pkl
from torch.utils.data import TensorDataset, DataLoader
import re
import torch
import numpy as np


import os

# import stanfordnlp

# Download the Arabic models for the neural pipeline
# stanfordnlp.download('ar', force=True)
# Build a neural pipeline using the Arabic models
# nlp = stanfordnlp.Pipeline(lang='ar')

# def split_arabic_sentences_with_stanfordnlp(corpus_text):
#     # Process the text
#     doc = nlp(corpus_text)

#     # Extract sentences from the doc
#     sentences = [sentence.text for sentence in doc.sentences]

#     return sentences

max_len=600

with open('/content/drive/MyDrive/NLP project/Arabic-text-to-diacritics/files/arabic_letters.pickle', 'rb') as file:
            ARABIC_LETTERS_LIST = pkl.load(file)
with open('/content/drive/MyDrive/NLP project/Arabic-text-to-diacritics/files/diacritics.pickle', 'rb') as file:
            DIACRITICS_LIST = pkl.load(file)

arabic_letters=[]
for letter in ARABIC_LETTERS_LIST:
    arabic_letters.append(letter[0])
arabic_letters.append(" ")

dicritics=[]
for letter in DIACRITICS_LIST:
    dicritics.append(letter[0])

classes = {
    'َ': 0,
    'ُ': 1,
    'ِ': 2,
    'ْ': 3,
    'ّ': 4,
    'ً': 5,
    'ٌ': 6,
    'ٍ': 7,
    'َّ': 8,
    'ُّ': 9,
    'ِّ': 10,
    'ًّ': 11,
    'ٌّ': 12,
    'ٍّ': 13,
    'ّّ': 14,
    "":15,
}

inverted_classes = {v: k for k, v in classes.items()}

##################### to delete #####################
def read_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

def write_to_file_second(dirctory,file_path, text):
    if not os.path.exists(dirctory):
        os.makedirs(dirctory)

    with open(dirctory+"/"+file_path, "a", encoding="utf-8") as file:
        lines=text
        for l in lines:
            file.write(l)
        file.write('\n')
def write_to_file_labels(dirctory,file_path, text):
    if not os.path.exists(dirctory):
        os.makedirs(dirctory)

    with open(dirctory+"/"+file_path, "a", encoding="utf-8") as file:
        lines=text
        for l in lines:
            file.write(inverted_classes[l])
        file.write('\n')

def write_to_file_string(dirctory,file_path, text):
    if not os.path.exists(dirctory):
        os.makedirs(dirctory)

    with open(dirctory+"/"+file_path, "a", encoding="utf-8") as file:
        lines=text
        file.write(lines)
        file.write('\n')
#####################################################

def preprocess(text):
         # Remove URLs
        text = re.sub(r"http[s|S]\S+", "", text,flags=re.MULTILINE)
        text = re.sub(r"www\S+", "", text,flags=re.MULTILINE)
        # Remove English letters
        text = re.sub(r"[A-Za-z]+", "", text,flags=re.MULTILINE)
        # Remove Kashida Arabic character
        text = re.sub(r"\u0640", "", text,flags=re.MULTILINE)
        # Add space before and after the numbers
        text = re.sub(r"(\d+)", r" \1 ", text,flags=re.MULTILINE)
        # removes SHIFT+J Arabic character
        text = re.sub(r"\u0691", "", text,flags=re.MULTILINE)
        # remove english numbers
        text = re.sub(r"[0-9]+", "", text,flags=re.MULTILINE)
        # remove arabic numbers
        text = re.sub(r"[٠-٩]+", "", text,flags=re.MULTILINE)
         # remove brackets
        # text = re.sub(r"\[.*?\]", "", text)
        # text = re.sub(r"\(.*?\)", "", text)
        return text

def split_text(text):
    text=text.split('.')
    # split text to sentences on all arabic sparatators

    data=[]
    for t in text:
        if(len(t)==0): continue
        if(len(t)<max_len):
            while(len(t)<max_len):
                 t+=" "
            data.append(t)
        if(len(t)>max_len):
            data.append(t[:max_len])
            supdata=t[max_len:]
            while(len(supdata)>max_len):
                data.append(supdata[:max_len])
                supdata=supdata[max_len:]
            if(len(supdata)<max_len):
                while(len(supdata)<max_len):
                    supdata+=" "
                data.append(supdata)
    return data

def get_data_labels(text):
    data=""
    labels=[]
    for i in range(len(text)):
            if(text[i] in arabic_letters and text[i]):
                data+=(text[i])
                if(i+1<len(text) and text[i+1] in dicritics):
                    if(i+2<len(text) and classes[text[i+1]]==4 and text[i+2] in dicritics):
                        labels.append(classes[text[i+1]+text[i+2]])
                        i+=2
                    else:
                        labels.append(classes[text[i+1]])
                        i+=1
                else:
                    labels.append(15)
    return data,labels

# def one_hot_encoding(text):
#     onehot_encoded=[]
#     for i in range(len(text)):
#          if text[i] in arabic_letters:
#             idx=arabic_letters.index(text[i])
#             encode=np.zeros(len(arabic_letters))
#             encode[idx]=1
#             onehot_encoded.append(encode)
#     onehot_encoded=torch.tensor(onehot_encoded)
#     return onehot_encoded

def encoding(text):
    idx=arabic_letters.index(text)
    encode=np.zeros(len(arabic_letters))
    encode[idx]=1
    return torch.tensor(encode,dtype=torch.float32).to(device)


def get_dataloader(encoded_data, encoding_labels,batch_size=1):
    # Create TensorDataset
    dataset = TensorDataset(encoded_data, encoding_labels)
    # Create DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return dataloader


def get_validation():
    text=read_text('/content/drive/MyDrive/NLP project/Arabic-text-to-diacritics/Dataset/val.txt')
    text=preprocess(text)
    # size=int(0.02*len(text))
    # text=text[:size]
    # text=split_text(text)
    # write_to_file_second("test","data.txt",text)
    # text=split_text(text)
    data=[]
    labels=[]
    for i in range (len(text)):
        if(text[i] in arabic_letters):
                data+=(text[i])
                if(i+1<len(text) and text[i+1] in dicritics):
                    if(i+2<len(text) and classes[text[i+1]]==4 and text[i+2] in dicritics):
                        labels.append(classes[text[i+1]+text[i+2]])
                        i+=2
                    else:
                        labels.append(classes[text[i+1]])
                        i+=1
                else:
                    labels.append(15)
        # else:
        #     data.append(text[i])
        #     labels.append(15)
    encoded_data = torch.empty(0, len(arabic_letters),dtype=torch.float32).to(device)

    for letter in data:
        if letter in arabic_letters:
            x = encoding(letter).unsqueeze(0)
        else:
            x=np.zeros((1,len(arabic_letters)))
            x=torch.tensor(x,dtype=torch.float32).to(device)
        encoded_data = torch.cat((encoded_data, x), 0)
    labels=torch.tensor(labels,dtype=torch.long).to(device)
    # print(encoded_data.shape)
    # print(labels.shape)
    dataloader=get_dataloader(encoded_data,labels)

    return dataloader

def get_data(path):
    text=read_text(path)
    text=preprocess(text)
    # size=int(0.02*len(text))
    # text=text[:size]
    print(len(text))

    text=split_text(text)

    # write_to_file_second("test","data.txt",text)
    # # get max length of sentence in text
    # maxdata=text.split('\n')

    # max_len=0
    # for t in maxdata:
    #     if(len(t)>max_len):
    #         max_len=len(t)
    # print(max_len)

    # split text to sentences on all arabic sparatators
    # text = split_arabic_sentences_with_stanfordnlp(text)

    # Filter out empty strings or whitespace-only sentences
    # text = [s.strip() for s in sentences if s.strip()]

    data=[]
    labels=[]
    for t in text:
        d=""
        l=[]
        d,l=get_data_labels(t)
        if(len(d)<max_len):
            while(len(d)<max_len):
                d+=(" ")
                l.append(15)
        else:
            d=d[:max_len]
            l=l[:max_len]
        data.append(d)
        labels.append(l)
    return data,labels

def get_features(data,labels):
    encoded_data = torch.empty(0, max_len, len(arabic_letters),dtype=torch.float32).to(device)
    for d in data:
        enc = torch.empty(0, len(arabic_letters),dtype=torch.float32).to(device)
        for letter in d:
            x = encoding(letter).unsqueeze(0)
            enc = torch.cat((enc, x), 0)
        encoded_data = torch.cat((encoded_data, enc.unsqueeze(0)), 0)
    # print(encoded_data.shape)
    encoding_labels=torch.tensor(labels,dtype=torch.long).to(device)
    # print(encoding_labels.shape)
    return encoded_data,encoding_labels

class DataSet():

    def __init__(self,path,batch_size=1) :
        print("Loading data...")
        data,labels=get_data(path)
        # now labels is list of list [[1,2,3,4,5,15,15,0],[1,2,3,4,5,15,15,0]]
        # data is list of string ['احمد','محمد']
        print("Extracting features...")
        data,labels=get_features(data,labels)
        # now the data and labels are tensor
        # data is tensor of shape (number of sentences,600,37)
        # labels is tensor of shape (number of sentences,600)
        print("Creating dataloader...")
        dataloader=get_dataloader(data,labels,batch_size)
        self.x=data
        self.y=labels
        self.dataloader=dataloader
        print("Done data creation !")

    def __len__(self):
         return len(self.y)
    def item(self,idx):
         return encoding(self.x[idx]),self.y[idx]
    def getdata(self):
        return self.dataloader






In [8]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np

class LSTM(nn.Module):
    def __init__(self, inp_vocab_size: int, hidden_dim: int = 256, seq_len: int = 600, num_classes: int = 16):
        super().__init__()
        self.lstm = nn.LSTM(inp_vocab_size, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # Output layer for 0 to 16 integers

    def forward(self, input_sequence: torch.Tensor):
        output, _ = self.lstm(input_sequence)
        output = self.fc(output.reshape(-1, output.size(-1)))
        return output

def train(train_dl, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(20):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            inputs, targets = inputs.to(device), targets.to(device)
            yhat = model(inputs)
            yhat = yhat.view(inputs.size(0), inputs.size(1), -1)
            yhat = yhat.view(-1, yhat.size(2))
            targets = targets.view(-1)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()
            print(f'epoch {epoch} batch {i} loss {loss.item()}')
    torch.save(model.state_dict(), 'model.pth')

def calculate_DER(actual_labels, predicted_labels):
    if not isinstance(actual_labels, torch.Tensor):
        actual_labels = torch.tensor(actual_labels)
    if not isinstance(predicted_labels, torch.Tensor):
        predicted_labels = torch.tensor(predicted_labels)

    if len(actual_labels) != len(predicted_labels):
        raise ValueError("Lengths of actual and predicted labels should match.")

    total_errors = torch.sum(actual_labels != predicted_labels)
    total_frames = len(actual_labels)
    DER = (1-(total_errors / total_frames)) * 100.0
    return DER.item()

def evaluate_model(test_dl, model):
    predictions, actuals = [], []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for i, (inputs, targets) in enumerate(test_dl):
        inputs, targets = inputs.to(device), targets.to(device)
        yhat = model(inputs)
        yhat = yhat.detach().cpu().numpy()
        yhat = yhat.reshape(-1, yhat.shape[-1])
        predicted_classes = np.argmax(yhat, axis=1)
        targets = targets.cpu().numpy().reshape(-1)
        predictions.extend(predicted_classes.tolist())
        actuals.extend(targets.tolist())
    acc = calculate_DER(np.array(actuals), np.array(predictions))
    return acc


In [ ]:
# from data import DataSet, get_validation
# from model import LSTM, train, evaluate_model

Traindata = DataSet("/content/drive/MyDrive/NLP project/Arabic-text-to-diacritics/Dataset/val.txt", batch_size=256)
Traindataloader = Traindata.getdata()
Validationdataloader=get_validation()

inp_vocab_size = 37
hidden_dim = 256
seq_len = 600
num_classes = 16

model = LSTM(inp_vocab_size, hidden_dim, seq_len, num_classes).to(device)
print("-------------------start training-------------------")
train(Traindataloader, model)

# Validation=DataSet('Dataset/val.txt',batch_size=1)
# Validationdataloader=Validation.getdata()
print("-------------------start evaluating-------------------")
acc = evaluate_model(Validationdataloader, model)
print("Accuracy: ", acc)

# validation=get_validation()


Loading data...
933760
Extracting features...
Creating dataloader...
Done data creation !
-------------------start training-------------------
epoch 0 batch 0 loss 2.796307325363159
epoch 0 batch 1 loss 2.73392653465271
epoch 0 batch 2 loss 2.667786121368408
epoch 0 batch 3 loss 2.5993845462799072
epoch 0 batch 4 loss 2.5216524600982666
epoch 0 batch 5 loss 2.417729139328003
epoch 0 batch 6 loss 2.2765519618988037
epoch 0 batch 7 loss 2.0500404834747314
epoch 0 batch 8 loss 1.5534729957580566
epoch 0 batch 9 loss 0.8863129615783691
epoch 0 batch 10 loss 1.0943711996078491
epoch 1 batch 0 loss 1.1191633939743042
epoch 1 batch 1 loss 0.9341135621070862
epoch 1 batch 2 loss 0.8137668967247009
epoch 1 batch 3 loss 0.8455584645271301
epoch 1 batch 4 loss 0.8830914497375488
epoch 1 batch 5 loss 0.8690981864929199
epoch 1 batch 6 loss 0.8569626808166504
epoch 1 batch 7 loss 0.8410628437995911
epoch 1 batch 8 loss 0.8376221060752869
epoch 1 batch 9 loss 0.7535815834999084
epoch 1 batch 10 loss